In [1]:
import pandas as pd

file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path)

print(df.head())

                  event_time event_type  product_id          category_id  \
0  2019-10-01 00:00:00+00:00       cart     5773203  1487580005134238553   
1  2019-10-01 00:00:03+00:00       cart     5773353  1487580005134238553   
2  2019-10-01 00:00:07+00:00       cart     5881589  2151191071051219817   
3  2019-10-01 00:00:07+00:00       cart     5723490  1487580005134238553   
4  2019-10-01 00:00:15+00:00       cart     5881449  1487580013522845895   

    brand  price    user_id                          user_session  \
0  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
1  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
2  lovely  13.48  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   
3  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
4  lovely   0.56  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   

           event_time_moscow  event_month  \
0  2019-10-01 03:00:00+03:00         10.0   
1  2019-10-01 03:00:03+03:00         1

In [2]:
import pandas as pd

# CSV 또는 Parquet 파일을 불러온 뒤 df에 저장했다고 가정
# 예: df = pd.read_csv('your_file.csv') 또는 pd.read_parquet('your_file.parquet')

#  1. price가 0인 행 제거
df = df[df['price'] > 0]

#  2. brand 컬럼에서 결측치 제거
df = df.dropna(subset=['brand'])

# 결과 확인 (선택사항)
print(df.shape)
print(df['price'].min())
print(df['brand'].isnull().sum())

(11932733, 11)
0.06
0


In [3]:
import pandas as pd

# 1. 이벤트 타입별 제품별 유저 수 집계
product_funnel = df.pivot_table(
    index='product_id',
    columns='event_type',
    values='user_key',
    aggfunc='nunique',
    fill_value=0
).reset_index()

# 2. 전환률 계산 (view → purchase 기준)
product_funnel['conversion_rate'] = product_funnel['purchase'] / product_funnel['view']
product_funnel['conversion_rate'] = product_funnel['conversion_rate'].round(4)

# 3. view 수 10 이상인 제품만 필터링 (신뢰도 확보용)
filtered = product_funnel[product_funnel['view'] >= 10]

# 4. 전환률 기준 상위 10개 제품 추출
top_10_products = filtered.sort_values(by='conversion_rate', ascending=False).head(10)

# 5. 결과 출력
print("🎯 전환률 높은 Top 10 제품:")
print(top_10_products[['product_id', 'view', 'purchase', 'conversion_rate']])

# 6. CSV 저장
top_10_products.to_csv('top10_product_conversion_rate.csv', index=False)
print("✅ 'top10_product_conversion_rate.csv' 저장 완료!")


🎯 전환률 높은 Top 10 제품:
event_type  product_id  view  purchase  conversion_rate
26266          5894916    28        84           3.0000
26261          5894911    35       105           3.0000
25682          5887054    25        64           2.5600
25857          5889485    50       127           2.5400
21076          5858993    30        76           2.5333
5577           5700041    46       116           2.5217
25686          5887058    22        53           2.4091
15877          5829282    21        49           2.3333
26259          5894909    76       169           2.2237
5576           5700040    58       125           2.1552
✅ 'top10_product_conversion_rate.csv' 저장 완료!


In [4]:
import pandas as pd

# 1. 이벤트별 제품별 유저 수 집계
product_funnel = df.pivot_table(
    index='product_id',
    columns='event_type',
    values='user_key',
    aggfunc='nunique',
    fill_value=0
).reset_index()

# 2. 전환률 계산 (view → purchase)
product_funnel['conversion_rate'] = product_funnel['purchase'] / product_funnel['view']
product_funnel['conversion_rate'] = product_funnel['conversion_rate'].round(4)

# 3. 이탈률 계산
product_funnel['dropoff_rate'] = (1 - product_funnel['conversion_rate']).round(4)

# 4. view 수 10 이상 제품 필터링 (신뢰도 확보용)
filtered = product_funnel[product_funnel['view'] >= 10]

# 5. 이탈률 기준 Top 10 제품 추출
top_10_drop_products = filtered.sort_values(by='dropoff_rate', ascending=False).head(10)

# 6. 결과 출력
print("이탈률 높은 Top 10 제품:")
print(top_10_drop_products[['product_id', 'view', 'purchase', 'conversion_rate', 'dropoff_rate']])

# 7. CSV 저장
top_10_drop_products.to_csv('top10_product_dropoff_rate.csv', index=False)
print("✅ 'top10_product_dropoff_rate.csv' 저장 완료!")


이탈률 높은 Top 10 제품:
event_type  product_id  view  purchase  conversion_rate  dropoff_rate
0                 3752    64         0              0.0           1.0
15403          5826865    10         0              0.0           1.0
15407          5826869    17         0              0.0           1.0
15414          5826876    11         0              0.0           1.0
15417          5826879    14         0              0.0           1.0
15420          5826882    16         0              0.0           1.0
15424          5826910    12         0              0.0           1.0
15432          5826918    21         0              0.0           1.0
15442          5826928    14         0              0.0           1.0
15446          5826932    11         0              0.0           1.0
✅ 'top10_product_dropoff_rate.csv' 저장 완료!


In [5]:
import pandas as pd

# 결과 저장 리스트
monthly_top_products = []

# 월별 반복
for month, month_df in df.groupby('event_month'):
    # 1. 제품별 유저 수 집계 (월 기준)
    product_funnel = month_df.pivot_table(
        index='product_id',
        columns='event_type',
        values='user_key',
        aggfunc='nunique',
        fill_value=0
    ).reset_index()

    # 2. 전환률 계산 (view → purchase)
    product_funnel['conversion_rate'] = product_funnel['purchase'] / product_funnel['view']
    product_funnel['conversion_rate'] = product_funnel['conversion_rate'].round(4)

    # 3. view 수 10 이상 필터링
    filtered = product_funnel[product_funnel['view'] >= 10].copy()
    filtered['event_month'] = month  # 월 정보 추가

    # 4. 상위 10개 제품만 추출
    top_10 = filtered.sort_values(by='conversion_rate', ascending=False).head(10)

    # 5. 결과 누적
    monthly_top_products.append(top_10)

# 6. 전체 월 결과 합치기
all_month_top_products = pd.concat(monthly_top_products, ignore_index=True)

# 7. 정렬 (월 → 전환률 높은 순)
all_month_top_products.sort_values(by=['event_month', 'conversion_rate'], 
                                   ascending=[True, False], inplace=True)

# 8. 저장
all_month_top_products.to_csv('monthly_top10_product_conversion.csv', index=False)
print("✅ 'monthly_top10_product_conversion.csv' 저장 완료!")

# 9. 미리보기: 각 월 상위 3개씩 출력
print("🎯 월별 전환률 상위 제품 미리보기:")
print(all_month_top_products.groupby('event_month').head(3))


✅ 'monthly_top10_product_conversion.csv' 저장 완료!
🎯 월별 전환률 상위 제품 미리보기:
event_type  product_id  cart  purchase  remove_from_cart  view  \
0              5889485    53        32                20    11   
1              5700050    84        40                47    16   
2              5894920    49        22                27    10   
10             5889485    67        30                20    11   
11             5883316    64        22                32    10   
12             5894907    61        35                30    16   
20             5887061    67        29                32    13   
21             5883316   106        42                41    19   
22             5659680    68        25                30    12   
30             5894918   111        41                60    10   
31             5894909   159        66                64    19   
32             5894911    81        33                31    10   
40             5894911    75        32                35    10   
41     

In [9]:
import pandas as pd

# 결과 저장 리스트
monthly_top_drop_products = []

# 월별 반복
for month, month_df in df.groupby('event_month'):
    # 1. 제품별 유저 수 집계 (해당 월 기준)
    product_funnel = month_df.pivot_table(
        index='product_id',
        columns='event_type',
        values='user_key',
        aggfunc='nunique',
        fill_value=0
    ).reset_index()

    # 2. 누락된 이벤트 처리
    for col in ['view', 'purchase']:
        if col not in product_funnel.columns:
            product_funnel[col] = 0

     # 3. 이탈률 계산
    product_funnel['conversion_rate'] = product_funnel['purchase'] / product_funnel['view']
    product_funnel['conversion_rate'] = product_funnel['conversion_rate'].round(4)
    product_funnel['dropoff_rate'] = (1 - product_funnel['conversion_rate']).round(4)

    # ✅ 4. purchase > 0 조건 추가
    filtered = product_funnel[(product_funnel['view'] >= 10) & (product_funnel['purchase'] > 0)].copy()
    filtered['event_month'] = month


    # 5. 이탈률 기준 상위 10개 제품 추출
    top_10 = filtered.sort_values(by='dropoff_rate', ascending=False).head(10)
    monthly_top_drop_products.append(top_10)

# 6. 전체 월 결과 병합
all_month_top_drop = pd.concat(monthly_top_drop_products, ignore_index=True)

# 7. 정렬: 월 오름차순, 이탈률 내림차순
all_month_top_drop.sort_values(by=['event_month', 'dropoff_rate'], 
                               ascending=[True, False], inplace=True)

# 8. 저장
all_month_top_drop.to_csv('monthly_top10_product_dropoff.csv', index=False)
print("✅ 'monthly_top10_product_dropoff.csv' 저장 완료!")

# 9. 미리보기
print("📉 월별 이탈률 높은 제품 (상위 3개씩):")
print(all_month_top_drop.groupby('event_month').head(3))



✅ 'monthly_top10_product_dropoff.csv' 저장 완료!
📉 월별 이탈률 높은 제품 (상위 3개씩):
event_type  product_id  cart  purchase  remove_from_cart   view  \
0              5915337    21         2                 1   4820   
1              5861591     6         1                38   1110   
2              5770029    22         3                12   3028   
10             5808664    55         4                 8  14783   
11             5805511    20         1                 4   1105   
12             5714240    16         1                12    628   
20             5817159    27         1                14    790   
21             5841298    25         1                16    425   
22             5877528    11         1                 4    339   
30             5771614    17         1                10   1217   
31             5770135    29         2                16   1411   
32             5888096    15         1                 3    546   
40             5770135    16         1                10   